In [1]:
import numpy as np 
import pandas as pd
import time
import random
import datetime
import gc
import pickle
import sklearn
from scipy import sparse
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.feature_selection import chi2, SelectPercentile,f_classif
import lightgbm as lgb
import warnings
from sklearn.externals import joblib 

warnings.filterwarnings("ignore")

D:\anaconda3411\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def memory_optimze(df):
    print("**********正在统计数据所占内存大小****************")
    memory_0 = df.memory_usage(deep="True").sum()/1024.0/1024.0
    print("*******处理int型***************")
    for i in df.select_dtypes(include=["integer"]).columns:
        if i !='instance_id':
            print(i)
            df[i] = pd.to_numeric(df[i],downcast="integer")
        
    print("******处理float型**************")
    for i in df.select_dtypes(include=["float"]).columns:
        df[i] = pd.to_numeric(df[i],downcast="float")
    print("**********处理object型（字符型）**************")

    memory_1 = df.memory_usage(deep="True").sum()/1024.0/1024.0

    print("优化前，数据集所占内存大小为{0}M".format(memory_0))
    print("优化后，数据集所占内存大小为{0}M".format(memory_1))
    print("优化后，内存减小了{0}%".format((memory_0-memory_1)/memory_0 * 100))
    return df

## 1.鱼佬+count+rank

#### 3.1数据准备

In [14]:
# fish_feature1 = pd.read_pickle("../round2NewFeature/feature_min_part1.pickle")
# fish_feature2 = pd.read_pickle("../round2NewFeature/feature_min_part2.pickle")
# fish_feature = pd.merge(fish_feature1,fish_feature2,on="instance_id",how="left")
# del fish_feature1
# del fish_feature2
# gc.collect()

# #------添加计数特征
# count_feature = pd.read_pickle("../round2NewFeature/genCountFeature.pickle")
# data = pd.merge(fish_feature,count_feature,on="instance_id",how='left')
# del count_feature
# gc.collect()

# rank_feature = pd.read_pickle("../round2NewFeature/rankFeature.pickle")
# data = pd.merge(data,rank_feature,on="instance_id",how='left')

# del rank_feature
# gc.collect()

# #添加groupby Feature
# groupby_feature = pd.read_pickle("../round2NewFeature/groupbyFeature.pickle")
# data = pd.merge(data,groupby_feature,on="instance_id",how='left')
# del groupby_feature
# gc.collect()
rank_columns = list(filter(lambda x:"rank" in str(x),data.columns.tolist()))
for i in rank_columns:
    data[i] = data[i].astype(int)
    data[i] = pd.to_numeric(data[i],downcast="integer")
data = memory_optimze(data)
print(data.shape)



**********正在统计数据所占内存大小****************
*******处理int型***************
city
province
carrier
devtype
make
model
nnt
os
osv
os_name
adid
advert_id
orderid
advert_industry_inner
campaign_id
creative_id
creative_tp_dnf
app_cate_id
f_channel
app_id
inner_slot_id
creative_type
creative_width
creative_height
creative_is_jump
creative_is_download
creative_has_deeplink
advert_name
click
day
hour
node1_industry
node2_industry
f1_channel
f2_channel
period
osv_os
devtype_make
advert_id_1
advert_industry_inner_1
advert_name_1
campaign_id_1
creative_height_1
creative_tp_dnf_1
creative_width_1
province_1
f_channel_1
devtype_1
0
3004430
3004434
3004059
3003779
3004214
3004081
3004500
3004506
3003851
nan
3004089
3003863
3003537
3003559
3003379
3004406
3003715
3003509
3004093
3004504
3003939
3004266
3003843
3004262
3003865
3003875
3003869
3004095
3004153
gd_2100000
3004147
3004149
3003679
gd_2100001
3003727
3003375
3003489
3003417
3004468
3003505
3004258
3004141
2100041
3003845
3004454
3003659
3004314
300

In [9]:
data.columns.tolist()

['instance_id',
 'city',
 'province',
 'carrier',
 'devtype',
 'make',
 'model',
 'nnt',
 'os',
 'osv',
 'os_name',
 'adid',
 'advert_id',
 'orderid',
 'advert_industry_inner',
 'campaign_id',
 'creative_id',
 'creative_tp_dnf',
 'app_cate_id',
 'f_channel',
 'app_id',
 'inner_slot_id',
 'creative_type',
 'creative_width',
 'creative_height',
 'creative_is_jump',
 'creative_is_download',
 'creative_has_deeplink',
 'advert_name',
 'click',
 'day',
 'hour',
 'node1_industry',
 'node2_industry',
 'f1_channel',
 'f2_channel',
 'period',
 'osv_os',
 'devtype_make',
 'advert_id_1',
 'advert_id_rate',
 'advert_industry_inner_1',
 'advert_industry_inner_rate',
 'advert_name_1',
 'advert_name_rate',
 'campaign_id_1',
 'campaign_id_rate',
 'creative_height_1',
 'creative_height_rate',
 'creative_tp_dnf_1',
 'creative_tp_dnf_rate',
 'creative_width_1',
 'creative_width_rate',
 'province_1',
 'province_rate',
 'f_channel_1',
 'f_channel_rate',
 'devtype_1',
 'devtype_rate',
 0,
 '3004430',
 '30044

In [15]:
data.to_pickle("../round2NewFeature/v1_modeling_data.pickle")
data = pd.read_pickle("../round2NewFeature/v1_modeling_data.pickle")
print(data.shape)
train = data[data["click"]!=-1]
test = data[data["click"]==-1]
del data
gc.collect()

(3072915, 161)


451

In [16]:
# train.to_pickle("../round2NewFeature/v1_train_data.pickle")
# test.to_pickle("../round2NewFeature/v1_test_data.pickle")

In [3]:
train = pd.read_pickle("../round2NewFeature/v1_train_data.pickle")
test = pd.read_pickle("../round2NewFeature/v1_test_data.pickle")

In [4]:
y_train = train["click"]
res = test.loc[:, ['instance_id']]

X_loc_train = train.drop(["instance_id","click"],axis=1).values
train_cols = train.drop(["instance_id","click"],axis=1).columns
y_loc_train = y_train.values
X_loc_test = test.drop(["instance_id","click"],axis=1).values
del train
del test
gc.collect()

14

In [ ]:
print("维度检查")
print(X_loc_train.shape,y_loc_train.shape,X_loc_test.shape)

#### 1.2 建模

In [ ]:
seed_ls = []
# 五折交叉训练，构造五个模型
skf=list(StratifiedKFold(y_loc_train, n_folds=5, shuffle=True, random_state=1024))
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf):
    print("Fold", i)
    random_seed = random.randint(42,3000)
    print("种子是",random_seed)
    seed_ls.append(random_seed)
    #------------------------------------#
    # 模型部分
    model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.05, n_estimators=2000,
                           max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                           min_child_weight=5, min_child_samples=10, subsample=0.8, subsample_freq=1,
                           colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=random_seed, n_jobs=15, silent=True)
    #------------------------------------#
    lgb_model = model.fit(X_loc_train[train_index], y_loc_train[train_index],
                          eval_names =['train','valid'],
                          eval_metric='logloss',
                          eval_set=[(X_loc_train[train_index], y_loc_train[train_index]), 
                                    (X_loc_train[test_index], y_loc_train[test_index])],early_stopping_rounds=100)
    baseloss.append(lgb_model.best_score_['valid']['binary_logloss'])
    loss += lgb_model.best_score_['valid']['binary_logloss']
    test_pred= lgb_model.predict_proba(X_loc_test, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    res['prob_%s' % str(i)] = test_pred
print('logloss:', baseloss, loss/5)

Fold 0
种子是 257
[1]	train's binary_logloss: 0.671437	valid's binary_logloss: 0.671446
Training until validation scores don't improve for 100 rounds.
[2]	train's binary_logloss: 0.651822	valid's binary_logloss: 0.651847
[3]	train's binary_logloss: 0.633989	valid's binary_logloss: 0.634021
[4]	train's binary_logloss: 0.617763	valid's binary_logloss: 0.617801
[5]	train's binary_logloss: 0.60298	valid's binary_logloss: 0.60303
[6]	train's binary_logloss: 0.589426	valid's binary_logloss: 0.589487
[7]	train's binary_logloss: 0.577041	valid's binary_logloss: 0.57711
[8]	train's binary_logloss: 0.565665	valid's binary_logloss: 0.565746
[9]	train's binary_logloss: 0.555203	valid's binary_logloss: 0.555289
[10]	train's binary_logloss: 0.545577	valid's binary_logloss: 0.545672
[11]	train's binary_logloss: 0.536677	valid's binary_logloss: 0.536778
[12]	train's binary_logloss: 0.528474	valid's binary_logloss: 0.528585
[13]	train's binary_logloss: 0.52091	valid's binary_logloss: 0.521035
[14]	train's

In [12]:
joblib.dump(model,"../round2NewResult/v1_model")

['../round2NewResult/v1_model']

In [13]:
fea_imp = pd.Series(model.feature_importances_,index=train_cols).sort_values(ascending=False)

In [16]:
fea_imp.to_csv("../round2NewResult/fea_imp_v1.csv",encoding="utf-8")

#### 1.3 预测

In [8]:
# 加权平均
res['predicted_score'] = 0
for i in range(5):
    res['predicted_score'] += res['prob_%s' % str(i)]
res['predicted_score'] = res['predicted_score']/5

# 提交结果
mean = res['predicted_score'].mean()
print('mean:',mean)
now = datetime.datetime.now()
now = now.strftime('%m-%d-%H-%M')
res[['instance_id', 'predicted_score']].to_csv("../round2NewResult/round2_v1_model%s.csv" % now, index=False)

mean: 0.20973844488738272


## v2.添加groupby_count

#### 4.1数据准备

In [18]:
data = pd.read_pickle("../round2NewFeature/v1_modeling_data.pickle")
groupbyFeat = pd.read_pickle("../round2NewFeature/groupbyFeature.pickle")
data = pd.merge(data,groupbyFeat,on="instance_id",how="left")
del groupbyFeat
gc.collect()

groupbyRank = pd.read_pickle("../round2NewFeature/groupby_rankFeature.pickle")
data = pd.merge(data,groupbyRank,on="instance_id",how="left")
del groupbyRank
gc.collect()

rank_columns = list(filter(lambda x:"rank" in str(x),data.columns.tolist()))
for i in rank_columns:
    data[i] = data[i].astype(int)
    data[i] = pd.to_numeric(data[i],downcast="integer")
data = memory_optimze(data)
print(data.shape)

**********正在统计数据所占内存大小****************
*******处理int型***************
city
province
carrier
devtype
make
model
nnt
os
osv
os_name
adid
advert_id
orderid
advert_industry_inner
campaign_id
creative_id
creative_tp_dnf
app_cate_id
f_channel
app_id
inner_slot_id
creative_type
creative_width
creative_height
creative_is_jump
creative_is_download
creative_has_deeplink
advert_name
click
day
hour
node1_industry
node2_industry
f1_channel
f2_channel
period
osv_os
devtype_make
advert_id_1
advert_industry_inner_1
advert_name_1
campaign_id_1
creative_height_1
creative_tp_dnf_1
creative_width_1
province_1
f_channel_1
devtype_1
0
3004430
3004434
3004059
3003779
3004214
3004081
3004500
3004506
3003851
nan
3004089
3003863
3003537
3003559
3003379
3004406
3003715
3003509
3004093
3004504
3003939
3004266
3003843
3004262
3003865
3003875
3003869
3004095
3004153
gd_2100000
3004147
3004149
3003679
gd_2100001
3003727
3003375
3003489
3003417
3004468
3003505
3004258
3004141
2100041
3003845
3004454
3003659
3004314
300

In [20]:
#data.to_pickle("../round2NewFeature/v2_modeling_data.pickle")
#data = pd.read_pickle("../round2NewFeature/v2_modeling_data.pickle")
print(data.shape)
train = data[data["click"]!=-1]
test = data[data["click"]==-1]
del data
gc.collect()

(3072915, 209)


7

In [3]:
# train.to_pickle("../round2NewFeature/v2_train_data.pickle")
# test.to_pickle("../round2NewFeature/v2_test_data.pickle")
train = pd.read_pickle("../round2NewFeature/v2_train_data.pickle")
test = pd.read_pickle("../round2NewFeature/v2_test_data.pickle")

#### v1特征重要性为0的特征

In [4]:
# # drop_cols = ['model_cnt_rank', 'f_channel_cnt_rank', 'creative_id_cnt_rank',
# #        'creative_tp_dnf_cnt_rank', 'f2_channel_cnt_rank', 'nnt_cnt_rank',
# #        'user_tags_cnt_rank', 'f1_channel_cnt_rank', 'orderid_cnt_rank',
# #        'os_name', 'province_cnt_rank', 'city_cnt_rank',
# #        'node2_industry_cnt_rank', 'campaign_id_cnt_rank', 'app_id_cnt_rank',
# #        '3003417', 'node2_industry_cnt', 'day_cnt_rank', '3003875',
# #        'adid_cnt_rank', 'advert_id_cnt_rank', 'inner_slot_id_cnt_rank',
# #        'advert_industry_inner_cnt_rank', 'node1_industry_cnt_rank',
# #        'f1_channel', '3003869', 'advert_name_cnt_rank', 'app_cate_id_cnt_rank',
# #        'creative_is_download', '3004149', 'hour_cnt_rank']
# train = train.drop(drop_cols,axis=1)
# test = test.drop(drop_cols,axis=1)

In [5]:
y_train = train["click"]
res = test.loc[:, ['instance_id']]

X_loc_train = train.drop(["instance_id","click"],axis=1).values
train_cols = train.drop(["instance_id","click"],axis=1).columns
y_loc_train = y_train.values
X_loc_test = test.drop(["instance_id","click"],axis=1).values
del train
del test
gc.collect()

14

In [6]:
print("维度检查")
print(X_loc_train.shape,y_loc_train.shape,X_loc_test.shape)

维度检查
(2992639, 207) (2992639,) (80276, 207)


#### 2.2 建模

In [ ]:
seed_ls = []
# 五折交叉训练，构造五个模型
skf=list(StratifiedKFold(y_loc_train, n_folds=5, shuffle=True, random_state=1024))
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf):
    print("Fold", i)
    random_seed = random.randint(42,3000)
    print("种子是",random_seed)
    seed_ls.append(random_seed)
    #------------------------------------#
    # 模型部分
    model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.04, n_estimators=2000,
                           max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                           min_child_weight=5, min_child_samples=10, subsample=0.8, subsample_freq=1,
                           colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=random_seed, n_jobs=8, silent=True)
    #------------------------------------#
    lgb_model = model.fit(X_loc_train[train_index], y_loc_train[train_index],
                          eval_names =['train','valid'],
                          eval_metric='logloss',
                          eval_set=[(X_loc_train[train_index], y_loc_train[train_index]), 
                                    (X_loc_train[test_index], y_loc_train[test_index])],early_stopping_rounds=100)
    baseloss.append(lgb_model.best_score_['valid']['binary_logloss'])
    loss += lgb_model.best_score_['valid']['binary_logloss']
    test_pred= lgb_model.predict_proba(X_loc_test, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    res['prob_%s' % str(i)] = test_pred
print('logloss:', baseloss, loss/5)

Fold 0
种子是 2553
[1]	train's binary_logloss: 0.675701	valid's binary_logloss: 0.675709
Training until validation scores don't improve for 100 rounds.
[2]	train's binary_logloss: 0.659597	valid's binary_logloss: 0.659615
[3]	train's binary_logloss: 0.64469	valid's binary_logloss: 0.644714
[4]	train's binary_logloss: 0.630888	valid's binary_logloss: 0.630918
[5]	train's binary_logloss: 0.618061	valid's binary_logloss: 0.618094
[6]	train's binary_logloss: 0.606146	valid's binary_logloss: 0.606184
[7]	train's binary_logloss: 0.595052	valid's binary_logloss: 0.595099
[8]	train's binary_logloss: 0.584708	valid's binary_logloss: 0.584762
[9]	train's binary_logloss: 0.575062	valid's binary_logloss: 0.575128
[10]	train's binary_logloss: 0.566044	valid's binary_logloss: 0.566118
[11]	train's binary_logloss: 0.557601	valid's binary_logloss: 0.557682
[12]	train's binary_logloss: 0.549709	valid's binary_logloss: 0.549798
[13]	train's binary_logloss: 0.542285	valid's binary_logloss: 0.54238
[14]	trai

In [15]:
print('logloss:', baseloss, loss/5)

logloss: [0.4149720117951277, 0.414924340011059, 0.4145613869693602, 0.4146069217325172, 0.41481198801954045] 0.4147753297055209


In [8]:
# 加权平均
res['predicted_score'] = 0
for i in range(5):
    res['predicted_score'] += res['prob_%s' % str(i)]
res['predicted_score'] = res['predicted_score']/5

# 提交结果
mean = res['predicted_score'].mean()
print('mean:',mean)
now = datetime.datetime.now()
now = now.strftime('%m-%d-%H-%M')
res[['instance_id', 'predicted_score']].to_csv("../round2NewResult/round2_v2_model%s.csv" % now, index=False)

mean: 0.21642842364008136


In [9]:
joblib.dump(model,"../round2NewResult/v2.model")

['../round2NewResult/v2.model']

In [10]:
fea_imp = pd.Series(model.feature_importances_,index=train_cols).sort_values(ascending=False)

In [14]:
fea_imp.to_csv("../round2NewResult/fea_imp_v2.csv",encoding="utf-8")

#### 2.3 调参数

In [17]:
seed_ls = []
# 五折交叉训练，构造五个模型
skf=list(StratifiedKFold(y_loc_train, n_folds=5, shuffle=True, random_state=1024))
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf):
    if i==1:
        break
    print("Fold", i)
    random_seed = random.randint(42,3000)
    print("种子是",random_seed)
    seed_ls.append(random_seed)
    #------------------------------------#
    # 模型部分
    model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.1, n_estimators=2000,
                           max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                           min_child_weight=5, min_child_samples=10, subsample=0.8, subsample_freq=1,
                           colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=random_seed, n_jobs=8, silent=True)
    #------------------------------------#
    lgb_model = model.fit(X_loc_train[train_index], y_loc_train[train_index],
                          eval_names =['train','valid'],
                          eval_metric='logloss',
                          eval_set=[(X_loc_train[train_index], y_loc_train[train_index]), 
                                    (X_loc_train[test_index], y_loc_train[test_index])],early_stopping_rounds=100)
    baseloss.append(lgb_model.best_score_['valid']['binary_logloss'])
    loss += lgb_model.best_score_['valid']['binary_logloss']
    test_pred= lgb_model.predict_proba(X_loc_test, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    res['prob_%s' % str(i)] = test_pred
print('logloss:', baseloss, loss/5)

Fold 0
种子是 498


MemoryError: 

#### V3 : 鱼老 + 排序 + 计数 +groupBy 特征 + groupBy重要特征

In [3]:
data = pd.read_pickle("../round2NewFeature/v2_modeling_data.pickle")

groupbyImpFeature = pd.read_pickle("../round2NewFeature/round2_importfeature_groupby.pickle")
data = data.merge(groupbyImpFeature,how="left",on="instance_id")
del groupbyImpFeature

data.to_pickle("../round2NewFeature/v3_modeling_data.pickle")
data = pd.read_pickle("../round2NewFeature/v3_modeling_data.pickle")
print(data.shape)
train = data[data["click"]!=-1]
test = data[data["click"]==-1]
del data
gc.collect()

(3072915, 215)


28

###### 特征筛选

In [14]:
fea_imp1= pd.read_csv("../round2NewResult/fea_imp_v1.csv",header=None)
fea_imp2= pd.read_csv("../round2NewResult/fea_imp_v2.csv",header=None)
fea_1_0 = fea_imp1[fea_imp1[1]==0][0].tolist()
fea_2_0 = fea_imp2[fea_imp2[1]==0][0].tolist()
fea_0 = list(set(fea_1_0)|set(fea_1_0))

In [17]:
train.to_pickle("../round2NewFeature/v3_train_data.pickle")
test.to_pickle("../round2NewFeature/v3_test_data.pickle")
# train = pd.read_pickle("../round2NewFeature/v1_train_data.pickle")
# test = pd.read_pickle("../round2NewFeature/v1_test_data.pickle")

In [18]:
train = train.drop(fea_0,axis=1)
test = test.drop(fea_0,axis=1)

y_train = train["click"]
res = test.loc[:, ['instance_id']]

X_loc_train = train.drop(["instance_id","click"],axis=1).values
train_cols = train.drop(["instance_id","click"],axis=1).columns
y_loc_train = y_train.values
X_loc_test = test.drop(["instance_id","click"],axis=1).values
del train
del test
gc.collect()

276

In [19]:
print("维度检查")
print(X_loc_train.shape,y_loc_train.shape,X_loc_test.shape)

维度检查
(2992639, 182) (2992639,) (80276, 182)


In [ ]:
seed_ls = []
# 五折交叉训练，构造五个模型
skf=list(StratifiedKFold(y_loc_train, n_folds=5, shuffle=True, random_state=1024))
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf):
    print("Fold", i)
    random_seed = random.randint(42,3000)
    print("种子是",random_seed)
    seed_ls.append(random_seed)
    #------------------------------------#
    # 模型部分
    model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.04, n_estimators=2000,
                           max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                           min_child_weight=5, min_child_samples=10, subsample=0.8, subsample_freq=1,
                           colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=random_seed, n_jobs=8, silent=True)
    #------------------------------------#
    lgb_model = model.fit(X_loc_train[train_index], y_loc_train[train_index],
                          eval_names =['train','valid'],
                          eval_metric='logloss',
                          eval_set=[(X_loc_train[train_index], y_loc_train[train_index]), 
                                    (X_loc_train[test_index], y_loc_train[test_index])],early_stopping_rounds=100)
    baseloss.append(lgb_model.best_score_['valid']['binary_logloss'])
    loss += lgb_model.best_score_['valid']['binary_logloss']
    test_pred= lgb_model.predict_proba(X_loc_test, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    res['prob_%s' % str(i)] = test_pred
print('logloss:', baseloss, loss/5)

Fold 0
种子是 992


In [21]:
print('logloss:', baseloss, loss/5)

logloss: [0.41492750016456803, 0.4148218898655316, 0.4145284223327966, 0.4145542959293309, 0.4147365492960393] 0.4147137315176533


In [22]:
# 加权平均
res['predicted_score'] = 0
for i in range(5):
    res['predicted_score'] += res['prob_%s' % str(i)]
res['predicted_score'] = res['predicted_score']/5

# 提交结果
mean = res['predicted_score'].mean()
print('mean:',mean)
now = datetime.datetime.now()
now = now.strftime('%m-%d-%H-%M')
res[['instance_id', 'predicted_score']].to_csv("../round2NewResult/round2_v3_model%s.csv" % now, index=False)

mean: 0.21637185223873012


In [23]:
joblib.dump(model,"../round2NewResult/v3.model")

['../round2NewResult/v3.model']

In [24]:
fea_imp = pd.Series(model.feature_importances_,index=train_cols).sort_values(ascending=False)

In [26]:
fea_imp.to_csv("../round2NewResult/fea_imp_v3.csv",encoding="utf-8")

In [28]:
fea_imp[fea_imp==0]

adid_model_cnt_rank                    0
adid_advert_industry_inner_cnt_rank    0
adid_hour_cnt_rank                     0
adid_node1_industry_cnt_rank           0
adid_f2_channel_cnt_rank               0
adid_province_cnt_rank                 0
adid_user_tags_cnt_rank                0
adid_advert_id_cnt_rank                0
adid_creative_tp_dnf_cnt               0
adid_creative_id_cnt                   0
adid_f1_channel_cnt_rank               0
adid_day_cnt_rank                      0
adid_orderid_cnt                       0
adid_node2_industry_cnt_rank           0
adid_inner_slot_id_cnt_rank            0
adid_orderid_cnt_rank                  0
adid_osv_cnt_rank                      0
adid_app_id_cnt_rank                   0
adid_f_channel_cnt_rank                0
adid_creative_tp_dnf_cnt_rank          0
adid_make_cnt_rank                     0
adid_nnt_cnt_rank                      0
adid_creative_id_cnt_rank              0
adid_city_cnt_rank                     0
adid_campaign_id

#### V4 = V1 + 加时间特征

In [ ]:
data = pd.read_pickle("../round2NewFeature/v1_modeling_data.pickle")

timeFeature = pd.read_pickle("../round2NewFeature/timeFeature.pickle")
data = data.merge(timeFeature,how="left",on="instance_id")
del timeFeature

data.to_pickle("../round2NewFeature/v4_modeling_data.pickle")
data = pd.read_pickle("../round2NewFeature/v4_modeling_data.pickle")
print(data.shape)
train = data[data["click"]!=-1]
test = data[data["click"]==-1]
del data
gc.collect()

In [2]:
# train.to_pickle("../round2NewFeature/v4_train_data.pickle")
# test.to_pickle("../round2NewFeature/v4_test_data.pickle")
train = pd.read_pickle("../round2NewFeature/v4_train_data.pickle")
test = pd.read_pickle("../round2NewFeature/v4_test_data.pickle")

In [4]:
# fea_imp1 = pd.read_csv("../round2NewResult/fea_imp_v1.csv",header=None)
# fea_imp2 = pd.read_csv("../round2NewResult/fea_imp_v2.csv",header=None)
# fea_imp3 = pd.read_csv("../round2NewResult/fea_imp_v3.csv",header=None)

# fea_1_0 = fea_imp1[fea_imp1[1]==0][0].tolist()
# fea_2_0 = fea_imp2[fea_imp2[1]==0][0].tolist()
# fea_3_0 = fea_imp3[fea_imp3[1]==0][0].tolist()
# fea_0 = list(set(fea_1_0)|set(fea_2_0)|set(fea_3_0))

In [5]:
# train = train.drop(fea_0,axis=1)
# test = test.drop(fea_0,axis=1)

In [3]:
y_train = train["click"]
res = test.loc[:, ['instance_id']]

X_loc_train = train.drop(["instance_id","click"],axis=1).values
train_cols = train.drop(["instance_id","click"],axis=1).columns
y_loc_train = y_train.values
X_loc_test = test.drop(["instance_id","click"],axis=1).values
del train
del test
gc.collect()

21

In [4]:
print("维度检查")
print(X_loc_train.shape,y_loc_train.shape,X_loc_test.shape)

维度检查
(2992639, 217) (2992639,) (80276, 217)


#### v4 训练

In [ ]:
seed_ls = []
# 五折交叉训练，构造五个模型
skf=list(StratifiedKFold(y_loc_train, n_folds=5, shuffle=True, random_state=1024))
baseloss = []
loss = 0
model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=80, max_depth=-1, learning_rate=0.05, n_estimators=2000,
                           min_data_in_leaf=400,
                           max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                           min_child_weight=5, min_child_samples=10, subsample=0.8, subsample_freq=1,
                           colsample_bytree=1, reg_alpha=3, reg_lambda=5, n_jobs=8, silent=True,seed=1000)
for i, (train_index, test_index) in enumerate(skf):
    lgb_model = model.fit(X_loc_train[train_index], y_loc_train[train_index],
                          eval_names =['train','valid'],
                          eval_metric='logloss',
                          eval_set=[(X_loc_train[train_index], y_loc_train[train_index]), 
                                    (X_loc_train[test_index], y_loc_train[test_index])],early_stopping_rounds=100)
    baseloss.append(lgb_model.best_score_['valid']['binary_logloss'])
    loss += lgb_model.best_score_['valid']['binary_logloss']
    test_pred= lgb_model.predict_proba(X_loc_test, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    res['prob_%s' % str(i)] = test_pred
print('logloss:', baseloss, loss/5)

[1]	train's binary_logloss: 0.671286	valid's binary_logloss: 0.671299
Training until validation scores don't improve for 100 rounds.
[2]	train's binary_logloss: 0.651529	valid's binary_logloss: 0.651556
[3]	train's binary_logloss: 0.633588	valid's binary_logloss: 0.63363
[4]	train's binary_logloss: 0.617272	valid's binary_logloss: 0.617326
[5]	train's binary_logloss: 0.602378	valid's binary_logloss: 0.602443
[6]	train's binary_logloss: 0.58875	valid's binary_logloss: 0.588827
[7]	train's binary_logloss: 0.576276	valid's binary_logloss: 0.576364
[8]	train's binary_logloss: 0.564833	valid's binary_logloss: 0.564932
[9]	train's binary_logloss: 0.554292	valid's binary_logloss: 0.554403
[10]	train's binary_logloss: 0.544579	valid's binary_logloss: 0.544699
[11]	train's binary_logloss: 0.535629	valid's binary_logloss: 0.535757
[12]	train's binary_logloss: 0.527381	valid's binary_logloss: 0.527521
[13]	train's binary_logloss: 0.519758	valid's binary_logloss: 0.519905
[14]	train's binary_loglo

In [6]:
fea_imp = pd.Series(lgb_model.feature_importances_,index=train_cols).sort_values(ascending=False)
fea_imp

model                             3432
model_cnt                         2870
city_cnt                          1994
city                              1985
to_adid_lasttime                  1586
province_rate                     1558
osv                               1296
spt                               1178
osv_cnt                           1134
make_cnt                          1096
adid_frequence                    1093
province_1                        1062
to_adid_mintime                   1015
to_adid_maxtime                    973
inner_slot_id                      962
to_inner_slot_id_lasttime          932
hour                               927
hour_cnt                           926
advert_id_frequence                925
adid_time_delta                    859
to_app_id_lasttime                 803
adid_maxtime_to_maxtime            792
province_cnt                       786
orderid                            707
app_id_frequence                   697
make                     

In [7]:
print('logloss:', baseloss, loss/5)

logloss: [0.4149774913309688, 0.4148787716556477, 0.41459275326838635, 0.41457975718920076, 0.41480551251120135] 0.414766857191081


In [8]:
# 加权平均
res['predicted_score'] = 0
for i in range(5):
    res['predicted_score'] += res['prob_%s' % str(i)]
res['predicted_score'] = res['predicted_score']/5

# 提交结果
mean = res['predicted_score'].mean()
print('mean:',mean)
now = datetime.datetime.now()
now = now.strftime('%m-%d-%H-%M')
res[['instance_id', 'predicted_score']].to_csv("../round2NewResult/round2_v4_model%s.csv" % now, index=False)

mean: 0.22211225371476878


### stacking